# Основы Pandas

Базы данных неотъемлимая часть любой системы, поэтому она должна быть адаптирована под любую задачу. Из всех популярных вариантов, наиболее подходящей к нашей работе оказался фреймворк <b>Pandas</b>, за счёт скорости работы, именовонности переменных и удобного восприятия записанных файлов без использования языков программирования

Из его основных возможностей выделим:<br>
1) Простая работа с .csv файлами
2) Производительность при фильтрации и сортировке хранилища
3) Интеграция с Python<br>

При этом стоит учесть неоптимизированное пересоздание хранилище при постоянном добавлении или удалении элементов, что явно продемонстрирую далее

## DataFrame и его структура

Библиотека работает со своим типом данных, названным <b>DataFrame</b>. Большая часть операций является методами данного класса. Для начала разберёмся, как создать такую переменную<br>
В первую очередь, импортируем всё с использованием сокращённого названия:

In [ ]:
import pandas as pd

По своей сути <b>DataFrame</b> - словарь, значениями которого являются массивы Numpy, о которых поговорим в следующем модуле. Основное удобство - наличие индексов, подобно спискам, которые распространяются на все столбцы. Таким образом, можем работать со словарём как с двумерной матрицей значений, сравнивая столбцы и строки по их заголовкам и индексам. Создадим пустой <b>df</b>:

In [ ]:
df = pd.DataFrame()
print(df)

Сразу видим пустые заголовки и явное сообщение, что хранилище пустое. Так как значения разбиваются по названиям, значит, формирование хранилища основанно на именнованных контейнерах - словарях

In [ ]:
df = pd.DataFrame({})
print(df, '\n')

df = pd.DataFrame({'a': [1, 2], 'b': [3, 4]})
print(df)

Передавая пустой словарь, получаем тот же результат. А вот с заполненным словарём картина совсем другая - отрисовалась матрица, столбцы которой точно повторяют переданный словарь, а строки проиндексированы с 0. Получается, мы можем попробовать обращаться к элементам матрицы по её "координатам"

In [ ]:
print(df['b'][0])
print(df['b'][1])

Первым элементом указываем ключ (название столбца), вторым - индекс в контейнере. То есть, аналогично словарю! 
Как измениться ситуация, если передать словарь со значением в виде числа

In [ ]:
df = pd.DataFrame({1: 2})

Не удалось создать индексы... А если так:

In [ ]:
df = pd.DataFrame([{1: 2}])
print(df)

Всё же pandas требует контейнеры в значениях, чтобы их можно было проиндексировать. Держим в голове эту особенность<br>
Покажу ещё один способ создания DF через ключи и значения:

In [ ]:
keys = ['column_1', 'column_2']
values = [[1, 2], [3, 4],[5, 6]]
print(pd.DataFrame(values, columns=keys))

Здесь мы создаём список списков, в которых хранится строка данных, а заголовки передаётся отдельным контейнером. Так удобно создавать пустые хранилища с разбиением по колонкам

На данный момент, мы видим, что новый тип данных точно не хуже стандартного словаря. Теперь перейдём к дополнительным возможностям. Во первых, мы можем проихвести сортировку по элементов по столбцам:

In [ ]:
df = pd.DataFrame({'Index': [4, 8, 3, 7, 1, 5, 2, 6], 'Name': ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']})

df = df.sort_values(by='Index')
print(df)

Метод `.sort_values(by, ignore_index=False, ascending=True, inplace=False, ...)` отвечает за сортировку. <i>ignore_index</i> при значении <i>True</i> перенумирует строки после сортировки, а <i>ascending</i> включит обратную сортировку при значении <i>False

In [ ]:
print(df, '\n')
df = df.sort_values(by='Name', ascending=False, ignore_index=True)
print(df)

Аргумент <i>inplace</i> делает замену на месте

In [ ]:
print(df, '\n')
df.sort_values(by='Index', ascending=False, ignore_index=True, inplace=True)
print(df)

В зависимости от указанной оси, можно производить перестановки как по строкам, так и по столбцам, но для первого будет нужен метод `.sort_index(axis=0, ignore_index=False, ascending=True, inplace=False, ...)`. Первый аргумент указывает ось сортировки {0: по индексам, 1: по названиям столбцов}, остальные аргументы аналогичны предыдущему методу

In [ ]:
df.sort_values(by='Index', inplace=True)
print(df, '\n')
df.sort_index(inplace=True)
print(df, '\n')
df.sort_index(axis=1, ascending=False, inplace=True)
print(df)

Более того, есть возможность производить сортировку по нескольким столбцам сразу. Для этого указываем их в виде списка

In [ ]:
data = {
    'Имя': ['Иван', 'Мария', 'Пётр', 'Анна'],
    'Возраст': [25, 30, 18, 20],
    'Пол': ['Мужской', 'Женский', 'Мужской', 'Женский'],
    'Хобби': [['Футбол', 'чтение'], ['Кулинария', 'путешествия'], ['Спорт', 'музыка'], ['Чтение', 'рисование']]
}

df = pd.DataFrame(data)
print(df, '\n')

print(df.sort_values(by=['Пол', 'Имя'], ignore_index=True))

## Получение элементов по условию

Нам редко понадобится весь объём данных. Поэтому должны научиться извлекать только то, что удовлетворяет определённым условиям, как минимум индексам. Для этого используется метод `.loc`, возвращающий переданные названия строк. Ему в квадратных скобках передаётся условие и, опционально, требуемый столбец:

In [ ]:
data = {
    'Имя': ['Иван', 'Мария', 'Пётр', 'Анна'],
    'Возраст': [25, 30, 18, 20],
    'Пол': ['Мужской', 'Женский', 'Мужской', 'Женский'],
    'Хобби': [['Футбол', 'чтение'], ['Кулинария', 'путешествия'], ['Спорт', 'музыка'], ['Чтение', 'рисование']]
}

df = pd.DataFrame(data)
print(df, '\n')

print(df['Возраст'] > 21, '\n')
print(df.loc[df['Возраст'] > 21], '\n')
print(df.loc[df['Возраст'] > 21, 'Имя'])

Результатом применения метода является DataFrame из булевых значений, по которому выбираются необходимые элементы из хранилища.<br>
Можно применить более короткую форму

In [ ]:
print(df[df['Возраст'] > 21], '\n')
print(df[df['Возраст'] > 21]['Имя'])

Условие может быть более сложным:

In [ ]:
print(df.loc[(df['Возраст'] > 21) | (df['Пол'] == 'Мужской')], '\n')
print(df.loc[(df['Возраст'] > 21) & (df['Пол'] == 'Мужской')], '\n')
print(df.loc[(df['Возраст'] >= 20) & (df['Пол'] != 'Мужской')])

Операцию извлечения значений в определённом диапазоне можно выполнить методом `.between(left, right, inclusive='both')`. Он возвращает DataFrame без лишних (не входящих в диапазон) данных в виде булевых значений

In [ ]:
data = {
    'Имя': ['Иван', 'Мария', 'Пётр', 'Анна'],
    'Возраст': [25, 30, 18, 20],
    'Пол': ['Мужской', 'Женский', 'Мужской', 'Женский'],
    'Хобби': [['Футбол', 'чтение'], ['Кулинария', 'путешествия'], ['Спорт', 'музыка'], ['Чтение', 'рисование']]
}

df = pd.DataFrame(data)
print(df, '\n')

filtered = df['Возраст'].between(20, 25)
print(filtered)

Так удобно обращаться к выделенным частям хранилища, например, выбрали данные по первому столбцу, а по другому строим график

In [ ]:
print(df[filtered], '\n')
print(df[filtered]['Имя'])

Всегда получаем DataFrame

## Операции между 2-мя DataFrame

### pd.merge

До сих пор мы создавали DataFrame из отдельного словаря. Очевидно, что каждый раз преобразовывать данные между типами неоптимально. Рассмотрим варианты объединения, стыковки и разбиения матриц<br>
Начнём с операций расширения. `pd.merge(left, right, how='inner', on=None, ...)` соединяет 2 DF на основе общих столбцов, указываемых в ключе <i>on</i><br>
<b>Для сохранения преобразования, записывайте его в переменную

In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40]
})
df2 = pd.DataFrame({
    'Имя': ['Петр', 'Анна', 'Сергей', 'Михаил'],
    'Рост': [180, 165, 175, 190]
})

pd.merge(df1, df2, on=['Имя'])

Получили данные только для совпадающих имён в обоих словарях. За это отвечает ключ <i>how</i>. Он может принимать значения ["left", "right", "inner", "outer", "cross"]. По умолчанию "inner", что и означает использование совпадающих значений объединения. Посмотрим все:

In [ ]:
print(pd.merge(df1, df2, on=['Имя'], how='inner'), '\n')
print(pd.merge(df1, df2, on=['Имя'], how='left'), '\n')
print(pd.merge(df1, df2, on=['Имя'], how='right'), '\n')
print(pd.merge(df1, df2, on=['Имя'], how='outer'))

"left", "right" задают столбец значений, который и будет в результате. Все неизвестные данные заменяются на <i>NaN</i>. "outer" - противоположность "inner", удивительно..., отображает все встречаемые ключи<br>
Что насчёт "cross"? Он отображает все возможные комбинации строк объединяемых DF:

In [ ]:
df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40],
    'Рост': [175, 180, 165, 170],
    'Вес': [75, 80, 65, 70]
})

df2 = pd.DataFrame({
    'Имя': ['Петр', 'Анна', 'Сергей', 'Михаил'],
    'Возраст': [30, 25, 40, 35],
    'Рост': [180, 165, 175, 190],
    'Зарплата': [50000, 45000, 60000, 55000]
})

pd.merge(df1, df2, how='cross')

### pd.concat

`pd.concat(df's, axis=0, join='outer', ignore_index=False, sort=False, keys=[])` объединяет DF вдоль выбранной оси<br>
<b>Для сохранения преобразования, записывайте его в переменную

In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40],
    'Рост': [175, 180, 165, 170],
    'Вес': [75, 80, 65, 70]
})

df2 = pd.DataFrame({
    'Имя': ['Петр', 'Анна', 'Сергей', 'Михаил'],
    'Возраст': [30, 25, 40, 35],
    'Рост': [180, 165, 175, 190],
    'Зарплата': [50000, 45000, 60000, 55000]
})

pd.concat([df1, df2])

Объединяемые элементы перечисляются в списке, чтобы они были единым аргументом. Так как по умолчанию ключ <i>join</i> принимает значение "outer", то все неизвестные значения заменены NaN

In [ ]:
pd.concat([df1, df2], join='inner', ignore_index=True)

Операцию можно выполнить вдоль другой оси, но в отличие от <b>pd.merge()</b>, ключи не объединяются

In [ ]:
pd.concat([df1, df2], axis=1)

Полезным элементом может оказаться добавление меток, разделяющих словари

In [ ]:
result = pd.concat([df1, df2], keys=['a', 'b'], ignore_index=False)
result

In [ ]:
result.loc['b']

### df.join

`.join(other, on, how='left', sort=False,...)` объединяет по индексам, аналогично `pd.merge`, но является методом, а не функцией

In [ ]:
import pandas as pd

df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40],
    'Рост': [175, 180, 165, 170]
})

df2 = pd.DataFrame({
    'Зарплата': [50000, 45000, 60000, 55000]
})

df1.join(df2)

Важно отметить: нужно избегать одинаково названных столбцов. Остальные аргументы работают аналогично `pd.merge`, так как внутри она же и вызывается!

In [ ]:
df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40],
    'Рост': [175, 180, 165, 170]
})

df2 = pd.DataFrame({
    'Зарплата': [50000, 45000, 60000]
})

df1.join(df2, how='inner')

In [ ]:
df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей', 'Валера'],
    'Возраст': [25, 30, 27, 40, 27],
    'Рост': [175, 180, 165, 170, 165]
})

df2 = pd.DataFrame({
    'Зарплата': [50000, 45000, 60000, 45000]
}, index=df1['Возраст'][:4])
print(df2)

df1.join(df2, on='Возраст')

В последнем примере <i>df2</i> задаётся с другими индексами, которые соответсвуют значениям из графы "Возраст", поэтому присоединение по ключу "Возраст", позволяет начислять зарплату по возрасту без необходимости прописывать её каждому участнику

### df.assign

Позволяет добавлять столбцы к DataFrame. Ключевым аргументом передаётся его название, а самим аргументом - значения

In [ ]:
df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей', 'Валера'],
    'Возраст': [25, 30, 27, 40, 27],
    'Рост': [175, 180, 165, 170, 165]
})

df1.assign(Зарплата=[50000, 45000, 60000, 45000, 50000])

### df.combine_first

Последнее объединение, что мы рассмотрим. `.combine_first(other)` дополняет DF, к которому применён метод, недостоющими значениями из второго

In [ ]:
import pandas as pd
from numpy import nan

df1 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [nan, 30, nan, 40],
    'Рост': [175, 180, 165, 170],
    'Вес': [75, 80, 65, 70]
})

df2 = pd.DataFrame({
    'Имя': ['Иван', 'Петр', 'Мария', 'Сергей'],
    'Возраст': [25, 30, 27, 40],
})

df1.combine_first(df2)

Отсутствующие данные возраста загрузились из другого DF

## df.iloc

Уже всречали схожый по названию метод `.loc()`, отличие в работе с индексами. Он позволяет получать строки по индексам и срезами разбить DF на отдельные части:

In [ ]:
import pandas as pd

data = {
    'Город': ['Москва', 'Санкт-Петербург', 'Казань', 'Нижний Новгород', 'Екатеринбург', 'Новосибирск', 'Красноярск', 'Самара'],
    'Население': [13, 6, 1, 2, 5, 4, 3, 7]
}

df = pd.DataFrame(data)
df.iloc[:5]

In [ ]:
df.iloc[:5:2]

## Группировка

Перейдём к более интересным вариантам. Представим эксперимент, в ходе которго некоторые параметры фиксируются (температура поле, ...). Значит, нас интересуют измерения при отдельных значениях данных постоянных. Метод `.groupby(by, ...)` сортирует df по группам, к которым можно применить ряд операций: суммирование элементов группы(`.sum()`), нахождение среднего(`.mean()`), их количества(`.count()`), минимума(`.min()`) и максимума(`.max()`)

In [ ]:
data = {'H': [1, 1, 1, 1, 2, 2, 2, 3],
        'T': [56, 56, 60, 60, 60, 60, 60, 76],
        'U': [0, 0, 0.1, 0.12, 0.5, 0.52, 0.54, 1.3]}
df = pd.DataFrame(data)
print(df, '\n')

print(df.groupby(['H', 'T']).mean(), '\n')
print(df.groupby(['H', 'T']).mean()['U'])

Применив метод к двум столбцам, получили результат операции для каждой вариации параметров. Обратите внимание - поле и температура стали индексами<br>
При необходимости можно применять несколько методов к одной группе или отдельно к столбцам методом `.agg()`:

In [ ]:
print(df.groupby('H').agg({'T': 'mean',
                           'U': 'max'}), '\n')
print(df.groupby('H').agg({'T': ['mean', 'min', 'max']}))

Здесь же упомянем об `.apply(func, axis=0)`, в котором можем передать функцию, применяемую к каждой группе. При <i>axis=1</i> можно выполнять построчные опреации:

In [ ]:
def error(group):
    return group['U'].max() - group['U'].min()

grouped = df.groupby(['H', 'T'])
print(grouped[['U']].apply(error))

df.apply(lambda row: row['H']*row['T'], axis=1)

## Маскировка

Порой нам требуется зафиксировать попадание значения в какой-то диапазон или выход из него, причём нам не важно насколько. В таких случаях пригодятся методы `.where(condition, other, inplace=False,...)`, `.mask(condition, other, inplace=False,...)`. Они проверяют каждое значение на соответсвие условию и в случаи несовпадения или совпадения соответственно заменяют его на NaN или указанное вторым аргументом число 

In [ ]:
data = {'H': [1, 1, 1, 1, 2, 2, 2, 3],
        'T': [56, 56, 60, 60, 60, 60, 60, 76],
        'U': [0, 0, 0.1, 0.12, 0.5, 0.52, 0.54, 1.3]}
df = pd.DataFrame(data)
print(df, '\n')

df.mask(df['U'] > 0.5, 1, inplace=True)
print(df, '\n')
df.where(df['U'] <= 0.5, 1, inplace=True)
print(df)